In [1]:
from face_detector import YoloDetector
import numpy as np
from PIL import Image
import torchvision.transforms.functional as TF
import os
import time
from tqdm.notebook import tqdm, trange
import torch
import copy
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision
import cv2
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
%matplotlib inline
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [2]:
facedt = YoloDetector(target_size=720,gpu=-1,min_face=90)

-1
/Users/e2318573/Documents/Projects/test/weights/yolov5n_state_dict.pt


In [3]:
class MyClassifier(nn.Module):
    def __init__(self, device):
        super().__init__()
        pretrained = models.vgg16(pretrained = True)
        pretrained.classifier.add_module('7',nn.Linear(1000,1))
        pretrained.classifier.add_module('8',nn.Sigmoid())
        self.layers = nn.ModuleList()
        self.layers.append(pretrained.features)
        self.layers.append(pretrained.avgpool)
        self.layers.append(nn.Flatten(end_dim = -1,start_dim=0))
        self.layers.append(pretrained.classifier)
    def forward(self,x):
        for layer in self.layers:
            x = layer(x)
        return x

In [4]:
cap = cv2.VideoCapture(0)
model = MyClassifier(device)

/Users/e2318573/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/e2318573/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
model = MyClassifier(device)
model.load_state_dict(torch.load('stored_dict/model.pth'))

<All keys matched successfully>

In [6]:
transform = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
])

In [6]:
ret, frame = cap.read()
boxes, points = facedt(frame)
x1,y1, x2, y2 = boxes[0][0]
temp = torch.from_numpy(frame)
temp = temp.permute([2, 0, 1])
image = TF.crop(temp, x1,y1, x2-x1, y2-y1)
print(image.size())
image = transforms.ToPILImage()(image)
image = transform(image)
image = image.type(torch.FloatTensor)
mean, std = image.mean([1,2]), image.std([1, 2])
torch.set_printoptions(precision=10)
print("mean: ",mean)
print("std: ", std)
image = transforms.Normalize(mean, std)(image)
outputs = model(image)
pred = torch.round(outputs)
pred = torch.squeeze(pred)
pred ='Distracted' if(pred == 1)  else 'Not Distracted'
print(pred)


torch.Size([3, 395, 442])
mean:  tensor([0.1879801005, 0.2019591480, 0.2132302076])
std:  tensor([0.2975057662, 0.3221705258, 0.3383626640])
Not Distracted


In [7]:
while True:
    ret, frame = cap.read()
    boxes, points = facedt(frame)
    for box in boxes[0]:
        x1,y1, x2, y2 = box
        cv2.rectangle(frame, (x1,y1), (x2,y2), (255, 0, 0),2)
        temp = torch.from_numpy(frame)
        temp = temp.permute([2, 0, 1])
        image = TF.crop(temp, x1,y1, x2-x1, y2-y1)
        #print(image.size())
        image = transforms.ToPILImage()(image)
        image = transform(image)
        image = image.type(torch.FloatTensor)
        mean, std = image.mean([1,2]), image.std([1, 2])
        # print("mean: ", mean)
        # print("std: ", std)
        try:
            image = transforms.Normalize(mean, std)(image)
            outputs = model(image)
            pred = torch.round(outputs)
            pred = torch.squeeze(pred)
            pred ='Distracted' if(pred == 1)  else 'Not Distracted'
            #print(pred)
            pred = torch.round(outputs)
            pred = torch.squeeze(pred)
            pred ='Distracted' if(pred == 1)  else 'Not Distracted'
            cv2.putText(frame, pred, (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0,), 3)
        except:
            print("error occured")

    cv2.imshow('Distraction Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error 